In [1]:
import pandas as pd
from dotenv import load_dotenv, find_dotenv
import os
from canopy.tokenizer import Tokenizer
from canopy.knowledge_base import KnowledgeBase
from canopy.models.data_models import Query
from canopy.context_engine import ContextEngine
from canopy.models.data_models import UserMessage,AssistantMessage
import gradio as gr

In [2]:
load_dotenv(find_dotenv())
gradio_port=os.getenv("PORT1")
debug_mode=os.getenv("CANOPY_DEBUG_INFO")
index_name = "pinecone-docs"
Tokenizer.initialize()

kb = KnowledgeBase(index_name)
context_engine = ContextEngine(kb)
from canopy.chat_engine import ChatEngine
chat_engine = ChatEngine(context_engine)
kb.connect()

In [3]:
def gradio_to_canopy(gradio_chatbot,message):
    canopy_list=[]
    last_message=UserMessage(content=message)
    for turn in gradio_chatbot:
        user_message, bot_message = turn
        canopy_list.append(UserMessage(content=user_message))
        canopy_list.append(AssistantMessage(content=bot_message))
    canopy_list.append(last_message)
    return canopy_list

def format_debug_window(bot_response):
    return f"model:{bot_response.model}\n\
    prompt tokens: {bot_response.usage.prompt_tokens}\n\
    completion tokens: {bot_response.usage.completion_tokens}\n\
    total tokens: {bot_response.usage.total_tokens}\n\
    context: {bot_response.debug_info}"


def respond(message, gradio_chatbot):
        canopy_chat_history=gradio_to_canopy(gradio_chatbot,message)
        chat_response=chat_engine.chat(canopy_chat_history)
        bot_response=chat_response.choices[0].message #This is a canopy object
        gradio_chatbot.append((message, bot_response.content))
        return "", gradio_chatbot,format_debug_window(chat_response)



In [4]:
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=240,show_copy_button=True) #just to fit the notebook
    msg = gr.Textbox(label="Prompt")
    btn = gr.Button("Enviar")
    clear = gr.ClearButton(components=[msg, chatbot], value="Limpiar")
    debug = gr.Textbox(label="Debug")
    btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot, debug])
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot, debug]) #Press enter to submit

gr.close_all()
demo.launch(share=False, server_port=int(os.environ['PORT1']))

Running on local URL:  http://127.0.0.1:8999

To create a public link, set `share=True` in `launch()`.


In [9]:
test_chat=[UserMessage(content="Tienes micro convertidores?")]
chat_response=chat_engine.chat(test_chat)
                              

In [5]:
str(chat_response)

'true'

In [6]:
if debug_mode:
    print("TRUE")

TRUE


In [11]:
context_obj=chat_engine._get_context(test_chat)
context_obj

Context(content=StuffingContextContent(root=[ContextQueryResult(query='Micro convertidores disponibles en el mercado', snippets=[ContextSnippet(source='https://contentia.cl/products/micro-convertidor-bidireccional-sdi-hdmi-3g-con-fuente-de-alimentacion', text='resistentes. Un puerto USB tipo C permite que el convertidor bidireccional SDI/HDMI se conecte fácilmente y sea alimentado por monitores de pantalla grande, computadoras portátiles, computadoras de escritorio o proyectores. También puede alimentar este convertidor utilizando la fuente de alimentación de 100-240 VCA incluida, completa con cuatro adaptadores de enchufe internacionales.'), ContextSnippet(source='https://contentia.cl/products/micro-convertidor-hdmi-a-sdi-3g-con-fuente-de-alimentacion', text='Súper compacto y resistente, el Micro Converter SDI a HDMI 3G de Blackmagic Design le permite usar monitores y televisores con entrada HDMI para monitorear la producción. Este convertidor compacto tiene una entrada SD/HD/3G-SDI, 

In [12]:
type(context_obj)

canopy.models.data_models.Context

In [19]:
context_obj.content.root[1].snippets[0]

ContextSnippet(source='https://contentia.cl/products/micro-convertidor-hdmi-a-sdi-3g', text='Súper compacto y resistente, el Micro Converter HDMI to SDI 3G de Blackmagic Design le permite vincular cámaras o computadoras HDMI a monitores o componentes SDI (BNC) de estilo profesional.\nTambién puede usar varias unidades para extender HDMI a largas distancias usando un cable SDI.\nEste convertidor compacto tiene una entrada HDMI y dos salidas 3G-SD/HD/SDI que se pueden configurar en el nivel A o B.\nDetecta automáticamente y hace coincidir su entrada HDMI con la salida SDI y es compatible con resoluciones de entrada/salida de hasta 1080p60.\nEl Micro Converter HDMI a SDI 3G cuenta con soporte de código de tiempo HDMI, sincronización SDI para mayor estabilidad en largas tiradas de cable y soporte para una amplia gama de resoluciones de transmisión profesional, incluido el estilo cinematográfico de 48 fps.Este convertidor de bolsillo cuenta con una carcasa de metal resistente con indicadore

In [17]:
context_obj.content.root[0]

ContextQueryResult(query='Micro convertidores disponibles en el mercado', snippets=[ContextSnippet(source='https://contentia.cl/products/micro-convertidor-bidireccional-sdi-hdmi-3g-con-fuente-de-alimentacion', text='resistentes. Un puerto USB tipo C permite que el convertidor bidireccional SDI/HDMI se conecte fácilmente y sea alimentado por monitores de pantalla grande, computadoras portátiles, computadoras de escritorio o proyectores. También puede alimentar este convertidor utilizando la fuente de alimentación de 100-240 VCA incluida, completa con cuatro adaptadores de enchufe internacionales.'), ContextSnippet(source='https://contentia.cl/products/micro-convertidor-hdmi-a-sdi-3g-con-fuente-de-alimentacion', text='Súper compacto y resistente, el Micro Converter SDI a HDMI 3G de Blackmagic Design le permite usar monitores y televisores con entrada HDMI para monitorear la producción. Este convertidor compacto tiene una entrada SD/HD/3G-SDI, una salida HDMI y una salida de bucle SD/HD/3